In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import sys
sys.path.append('/Users/mregnier/Desktop/PhD Regnier/mypackages')


import Acquisition as Acq

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# FG-Buster packages
import component_model as c
import mixing_matrix as mm

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils

from qubic import SpectroImLib as sp
from importlib import reload
import gc
import copy
from pyoperators import *
import myPCG as mypcg


# PyOperators packages
from pyoperators import (
    BlockColumnOperator, BlockDiagonalOperator, BlockRowOperator,
    CompositionOperator, DiagonalOperator, I, IdentityOperator,
    MPIDistributionIdentityOperator, MPI, proxy_group, ReshapeOperator,
    rule_manager, pcg, Operator)

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")
# %matplotlib inline

## Now QUBIC and Planck acquisitions used for reconstruction

def integration(nu_min, nu_max, Nintegr, sky_config, d, fwhm_subbands = None):
    print(f'Integration from {nu_min:.2f} to {nu_max:.2f} GHz with {Nintegr} steps')
    obj = Acq.QubicIntegrated(d, Nsub=Nintegr, Nrec=Nintegr)
    if Nintegr == 1:
        allnus = np.array([np.mean([nu_min, nu_max])])
    else:
        allnus = np.linspace(nu_min, nu_max, Nintegr)
    m = obj.get_PySM_maps(sky_config, nus=allnus)
    
    if fwhm_subbands is not None:
        for i in range(Nintegr):
            C = HealpixConvolutionGaussianOperator(fwhm=fwhm_subbands[i])
            m[i] = C(m[i])
    
    return np.array([np.mean(m, axis=0)])


# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import pickle

with open('sol_with_noise.pickle', 'rb') as f:
    solnoisy, okpix = pickle.load(f)

In [ ]:
with open('sol_without_noise.pickle', 'rb') as f:
    solnoiseless, okpix = pickle.load(f)

In [ ]:
residuals = solnoisy - solnoiseless

In [ ]:
center = np.array([270, 0])
nside = 256
stk = ['I', 'Q', 'U']

for s in range(3):
    plt.figure(figsize=(9, 15))  
    if s==0:
        v = 1000
        r = 3
    else:
        v = 30
        r = 5
    for i in range(5):
        mymap_noisy = np.zeros(12*nside**2) + hp.UNSEEN
        mymap_noiseless = np.zeros(12*nside**2) + hp.UNSEEN
        mymap_noisy[okpix] = solnoisy[i,:,s]
        mymap_noiseless[okpix] = solnoiseless[i,:,s]
        mydiff = mymap_noisy - mymap_noiseless
        mydiff[~okpix] = hp.UNSEEN
        hp.gnomview(mymap_noiseless, rot=center, sub=(5, 3, 3*i+1), title='Input {} sub {}/{}'.format(stk[s], i+1, 5), min=-v, max=v, reso=15, cmap='jet')
        hp.gnomview(mymap_noisy, rot=center, sub=(5, 3, 3*i+2), title='Output {} sub {}/{}'.format(stk[s], i+1, 5), min=-v, max=v, reso=15, cmap='jet')
        hp.gnomview(mydiff, rot=center, sub=(5, 3, 3*i+3), title='Diff {0:} sub {1:}/{2:}'.format(stk[s], i+1, 5), min=-r, max=r, reso=15, cmap='jet')
    tight_layout()
  

In [ ]:
residuals.shape

In [ ]:
for i in range(1, np.shape(residuals)[0]):
    plot(residuals[0,:,0], residuals[i,:,0], ',')


In [ ]:
i=0
mycov = np.cov(residuals[:,:,i])
print(mycov)
print()
w, v = np.linalg.eigh(mycov)
print(w)
print()
print(v)
subplot(1,2,1)
plot(w, 'ko')
yscale('log')
subplot(1,2,2)
for i in range(len(w)):
    plot(v[:,i], 'o-', label=i)
legend()

In [ ]:
mycov = np.zeros((3,5,5))
rc('font', size=10)
rc('figure', figsize=(20, 5))
for i in range(3):
    subplot(1,3,i+1)
    mycov[i,:,:] = np.cov(residuals[:,:,i])
    imshow(mycov[i,:,:])
    print()
    print(mycov[i,:,:])
    title(stk[i])
    colorbar()
tight_layout()

In [ ]:
subplot(1,2,1)
for i in range(3):
    plot(np.diag(mycov[i,:,:]), 'o-', label=stk[i])
legend()
ylim(0,3.5)

subplot(1,2,2)
for i in range(3):
    plot(np.diag(mycov[i,:,:]) / np.diag(mycov[i,:,:])[0], 'o-', label=stk[i])
legend()


In [ ]:
rc('font', size=10)
rc('figure', figsize=(20, 5))
for i in range(3):
    subplot(1,3,i+1)
    imshow(np.corrcoef(residuals[:,:,2]), vmin=0, vmax=1)
    print()
    print(np.corrcoef(residuals[:,:,2]))
    title(stk[i])
    colorbar()
tight_layout()

In [ ]:
residuals.shape

In [ ]:
res = np.transpose(residuals, axes=(2,0,1))
res.shape

In [ ]:
bla = np.reshape(res, (15, 27107))
imshow(np.corrcoef(bla), cmap='jet')
colorbar()

In [ ]:
bla = np.reshape(res, (15, 27107))
imshow(np.log10(np.corrcoef(bla)), vmin=-2, vmax=0, cmap='jet')
colorbar()